In [20]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import PIL
from PIL import Image
from skimage.color import rgb2gray
from scipy import ndimage as ndi
import cv2
import os
from os import listdir
from sklearn.utils import shuffle
import pickle
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint


In [2]:
directory_root = "../input/plant-images/Plant_Disease_Images/color_image"
print(len(listdir(directory_root)))

38


In [3]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        print(f"[INFO] Processing {plant_folder} ...")
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
                
        for single_plant_disease_image in plant_disease_folder_list :
            if single_plant_disease_image == ".DS_Store" :
                plant_disease_folder_list.remove(single_plant_disease_image)

        for image in plant_disease_folder_list:
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(image_directory)
                label_list.append(plant_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Tomato___Late_blight ...
[INFO] Processing Tomato___healthy ...
[INFO] Processing Grape___healthy ...
[INFO] Processing Orange___Haunglongbing_(Citrus_greening) ...
[INFO] Processing Soybean___healthy ...
[INFO] Processing Squash___Powdery_mildew ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight ...
[INFO] Processing Tomato___Early_blight ...
[INFO] Processing Tomato___Septoria_leaf_spot ...
[INFO] Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot ...
[INFO] Processing Strawberry___Leaf_scorch ...
[INFO] Processing Peach___healthy ...
[INFO] Processing Apple___Apple_scab ...
[INFO] Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus ...
[INFO] Processing Tomato___Bacterial_spot ...
[INFO] Processing Apple___Black_rot ...
[INFO] Processing Blueberry___healthy ...
[INFO] Processing Cherry_(including_sour)___Powdery_mildew ...
[INFO] Processing Peach___Bacterial_spot ...
[INFO] Processing 

In [4]:
img_info = pd.DataFrame({'image_path':image_list,'label':label_list})
print(img_info.head())
print(len(img_info))

                                          image_path                 label
0  ../input/plant-images/Plant_Disease_Images/col...  Tomato___Late_blight
1  ../input/plant-images/Plant_Disease_Images/col...  Tomato___Late_blight
2  ../input/plant-images/Plant_Disease_Images/col...  Tomato___Late_blight
3  ../input/plant-images/Plant_Disease_Images/col...  Tomato___Late_blight
4  ../input/plant-images/Plant_Disease_Images/col...  Tomato___Late_blight
54303


In [5]:
#new column (empty)
img_info["labels_integer"] = None
#index of new column
index_labels_integer = img_info.columns.get_loc("labels_integer")
#index of species column
index_species = img_info.columns.get_loc("label")
#to assign numeric labels starting with 0 for the first species
k = 0 
for i in range(len(img_info)):
    if i == 0:
        img_info.iloc[i, index_labels_integer] = k #here, k == 0
    if i > 0:
        if img_info.iloc[i-1, index_species] == img_info.iloc[i, index_species]:
            img_info.iloc[i, index_labels_integer] = k
        else:
            k += 1
            img_info.iloc[i, index_labels_integer] = k
img_info.tail()

,image_path,label,labels_integer
54298,../input/plant-images/Plant_Disease_Images/col...,Corn_(maize)___healthy,37
54299,../input/plant-images/Plant_Disease_Images/col...,Corn_(maize)___healthy,37
54300,../input/plant-images/Plant_Disease_Images/col...,Corn_(maize)___healthy,37
54301,../input/plant-images/Plant_Disease_Images/col...,Corn_(maize)___healthy,37
54302,../input/plant-images/Plant_Disease_Images/col...,Corn_(maize)___healthy,37


In [6]:
img_info = shuffle(img_info)
list_vectors = []

for image_path in img_info.image_path:
    #read as rgb array
    img = Image.open(image_path)
    size = (64, 64)
    img = img.resize(size, PIL.Image.ANTIALIAS)  
    img_array = np.array(img)
    #append image vector to list
    list_vectors.append(img_array)
    
print(len(list_vectors))

54303


In [7]:
X = np.stack((list_vectors))
Y =  img_info['labels_integer']
print(X.shape)

(54303, 64, 64, 3)


In [8]:
X = X/255
Y_one_hot = keras.utils.to_categorical(Y, num_classes=38)
print(Y.shape, Y_one_hot.shape)

(54303,) (54303, 38)


In [9]:
np.savez("x_images_arrayscnn", X)
np.savez("y_numeric_labelscnn", Y_one_hot)

In [10]:
x_npz = np.load("x_images_arrayscnn.npz")
X = x_npz['arr_0']

y_npz = np.load("y_numeric_labelscnn.npz")
Y_one_hot = y_npz['arr_0']

print(X.shape)

(54303, 64, 64, 3)


In [11]:
split_train = 0.8 #train 0.8, validate 0.1, test 0.1
split_val = 0.9
index_train = int(split_train*len(X))
index_val = int(split_val*len(X))

X_train = X[:index_train]
X_val = X[index_train:index_val]
X_test = X[index_val:]

Y_train = Y_one_hot[:index_train]
Y_val = Y_one_hot[index_train:index_val]
Y_test = Y_one_hot[index_val:]


print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(43442, 64, 64, 3) (5430, 64, 64, 3) (5431, 64, 64, 3) (43442, 38) (5430, 38) (5431, 38)


In [12]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]) 
num_classes = 38

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer=keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-8, amsgrad=False),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 60, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
activation_2 (Activation)    (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 64)        0         
__________

In [14]:
best_model_file = "disease_selected.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model...')
results = model.fit(X_train, Y_train, epochs=512, batch_size=64, validation_data=(X_val, Y_val), callbacks=[best_model])
print('Traing finished.')

print('Loading the best model...')
model = load_model(best_model_file)
print('Best Model loaded!')

Training model...
Instructions for updating:
Use tf.cast instead.
Train on 43442 samples, validate on 5430 samples
Epoch 1/512
43442/43442 [==============================] - 14s 333us/step - loss: 1.7852 - acc: 0.5076 - val_loss: 0.8694 - val_acc: 0.7556

Epoch 00001: val_loss improved from inf to 0.86941, saving model to disease_selected.h5
Epoch 2/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.9549 - acc: 0.7154 - val_loss: 0.5899 - val_acc: 0.8273

Epoch 00002: val_loss improved from 0.86941 to 0.58988, saving model to disease_selected.h5
Epoch 3/512
43442/43442 [==============================] - 10s 219us/step - loss: 0.7097 - acc: 0.7807 - val_loss: 0.4496 - val_acc: 0.8650

Epoch 00003: val_loss improved from 0.58988 to 0.44956, saving model to disease_selected.h5
Epoch 4/512
43442/43442 [==============================] - 9s 210us/step - loss: 0.5856 - acc: 0.8149 - val_loss: 0.3929 - val_acc: 0.8788

Epoch 00004: val_loss improved from 0.44956 to 0.39

43442/43442 [==============================] - 10s 221us/step - loss: 0.1176 - acc: 0.9599 - val_loss: 0.1399 - val_acc: 0.9580

Epoch 00037: val_loss did not improve from 0.12560
Epoch 38/512
43442/43442 [==============================] - 9s 210us/step - loss: 0.1218 - acc: 0.9602 - val_loss: 0.1386 - val_acc: 0.9615

Epoch 00038: val_loss did not improve from 0.12560
Epoch 39/512
43442/43442 [==============================] - 9s 210us/step - loss: 0.1106 - acc: 0.9625 - val_loss: 0.1209 - val_acc: 0.9630

Epoch 00039: val_loss improved from 0.12560 to 0.12091, saving model to disease_selected.h5
Epoch 40/512
43442/43442 [==============================] - 9s 211us/step - loss: 0.1076 - acc: 0.9636 - val_loss: 0.1171 - val_acc: 0.9661

Epoch 00040: val_loss improved from 0.12091 to 0.11707, saving model to disease_selected.h5
Epoch 41/512
43442/43442 [==============================] - 9s 218us/step - loss: 0.1132 - acc: 0.9615 - val_loss: 0.1191 - val_acc: 0.9657

Epoch 00041: val_loss

43442/43442 [==============================] - 9s 212us/step - loss: 0.0694 - acc: 0.9770 - val_loss: 0.1223 - val_acc: 0.9670

Epoch 00078: val_loss did not improve from 0.10436
Epoch 79/512
43442/43442 [==============================] - 9s 218us/step - loss: 0.0750 - acc: 0.9756 - val_loss: 0.1323 - val_acc: 0.9643

Epoch 00079: val_loss did not improve from 0.10436
Epoch 80/512
43442/43442 [==============================] - 9s 212us/step - loss: 0.0704 - acc: 0.9771 - val_loss: 0.1142 - val_acc: 0.9696

Epoch 00080: val_loss did not improve from 0.10436
Epoch 81/512
43442/43442 [==============================] - 9s 212us/step - loss: 0.0729 - acc: 0.9762 - val_loss: 0.1129 - val_acc: 0.9676

Epoch 00081: val_loss did not improve from 0.10436
Epoch 82/512
43442/43442 [==============================] - 10s 220us/step - loss: 0.0714 - acc: 0.9768 - val_loss: 0.1141 - val_acc: 0.9731

Epoch 00082: val_loss did not improve from 0.10436
Epoch 83/512
43442/43442 [==========================

43442/43442 [==============================] - 9s 212us/step - loss: 0.0577 - acc: 0.9815 - val_loss: 0.1070 - val_acc: 0.9738

Epoch 00120: val_loss did not improve from 0.10056
Epoch 121/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.0614 - acc: 0.9812 - val_loss: 0.1073 - val_acc: 0.9726

Epoch 00121: val_loss did not improve from 0.10056
Epoch 122/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.0574 - acc: 0.9820 - val_loss: 0.1165 - val_acc: 0.9729

Epoch 00122: val_loss did not improve from 0.10056
Epoch 123/512
43442/43442 [==============================] - 9s 211us/step - loss: 0.0560 - acc: 0.9818 - val_loss: 0.1145 - val_acc: 0.9718

Epoch 00123: val_loss did not improve from 0.10056
Epoch 124/512
43442/43442 [==============================] - 10s 220us/step - loss: 0.0547 - acc: 0.9832 - val_loss: 0.1275 - val_acc: 0.9700

Epoch 00124: val_loss did not improve from 0.10056
Epoch 125/512
43442/43442 [=====================


Epoch 00162: val_loss did not improve from 0.10056
Epoch 163/512
43442/43442 [==============================] - 9s 211us/step - loss: 0.0518 - acc: 0.9838 - val_loss: 0.1087 - val_acc: 0.9753

Epoch 00163: val_loss did not improve from 0.10056
Epoch 164/512
43442/43442 [==============================] - 9s 211us/step - loss: 0.0520 - acc: 0.9840 - val_loss: 0.1162 - val_acc: 0.9724

Epoch 00164: val_loss did not improve from 0.10056
Epoch 165/512
43442/43442 [==============================] - 9s 207us/step - loss: 0.0556 - acc: 0.9831 - val_loss: 0.1162 - val_acc: 0.9731

Epoch 00165: val_loss did not improve from 0.10056
Epoch 166/512
43442/43442 [==============================] - 9s 217us/step - loss: 0.0530 - acc: 0.9827 - val_loss: 0.1113 - val_acc: 0.9729

Epoch 00166: val_loss did not improve from 0.10056
Epoch 167/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.0495 - acc: 0.9843 - val_loss: 0.1112 - val_acc: 0.9744

Epoch 00167: val_loss did not impr

...

43442/43442 [==============================] - 9s 215us/step - loss: 0.0441 - acc: 0.9882 - val_loss: 0.1400 - val_acc: 0.9729

Epoch 00332: val_loss did not improve from 0.09891
Epoch 333/512
43442/43442 [==============================] - 9s 212us/step - loss: 0.0418 - acc: 0.9891 - val_loss: 0.1203 - val_acc: 0.9779

Epoch 00333: val_loss did not improve from 0.09891
Epoch 334/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.0460 - acc: 0.9874 - val_loss: 0.1149 - val_acc: 0.9740

Epoch 00334: val_loss did not improve from 0.09891
Epoch 335/512
43442/43442 [==============================] - 10s 223us/step - loss: 0.0446 - acc: 0.9885 - val_loss: 0.1262 - val_acc: 0.9783

Epoch 00335: val_loss did not improve from 0.09891
Epoch 336/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0443 - acc: 0.9886 - val_loss: 0.1242 - val_acc: 0.9779

Epoch 00336: val_loss did not improve from 0.09891
Epoch 337/512
43442/43442 [=====================

43442/43442 [==============================] - 9s 212us/step - loss: 0.0405 - acc: 0.9899 - val_loss: 0.1247 - val_acc: 0.9761

Epoch 00374: val_loss did not improve from 0.09891
Epoch 375/512
43442/43442 [==============================] - 9s 215us/step - loss: 0.0486 - acc: 0.9881 - val_loss: 0.1420 - val_acc: 0.9705

Epoch 00375: val_loss did not improve from 0.09891
Epoch 376/512
43442/43442 [==============================] - 10s 222us/step - loss: 0.0400 - acc: 0.9889 - val_loss: 0.1463 - val_acc: 0.9716

Epoch 00376: val_loss did not improve from 0.09891
Epoch 377/512
43442/43442 [==============================] - 9s 215us/step - loss: 0.0442 - acc: 0.9891 - val_loss: 0.1367 - val_acc: 0.9737

Epoch 00377: val_loss did not improve from 0.09891
Epoch 378/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0434 - acc: 0.9884 - val_loss: 0.1256 - val_acc: 0.9773

Epoch 00378: val_loss did not improve from 0.09891
Epoch 379/512
43442/43442 [=====================

43442/43442 [==============================] - 9s 213us/step - loss: 0.0444 - acc: 0.9897 - val_loss: 0.1167 - val_acc: 0.9768

Epoch 00416: val_loss did not improve from 0.09891
Epoch 417/512
43442/43442 [==============================] - 10s 221us/step - loss: 0.0473 - acc: 0.9888 - val_loss: 0.1320 - val_acc: 0.9775

Epoch 00417: val_loss did not improve from 0.09891
Epoch 418/512
43442/43442 [==============================] - 10s 220us/step - loss: 0.0448 - acc: 0.9891 - val_loss: 0.1238 - val_acc: 0.9755

Epoch 00418: val_loss did not improve from 0.09891
Epoch 419/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0389 - acc: 0.9903 - val_loss: 0.1228 - val_acc: 0.9773

Epoch 00419: val_loss did not improve from 0.09891
Epoch 420/512
43442/43442 [==============================] - 10s 221us/step - loss: 0.0482 - acc: 0.9880 - val_loss: 0.1243 - val_acc: 0.9773

Epoch 00420: val_loss did not improve from 0.09891
Epoch 421/512
43442/43442 [===================

43442/43442 [==============================] - 10s 219us/step - loss: 0.0450 - acc: 0.9887 - val_loss: 0.1307 - val_acc: 0.9772

Epoch 00458: val_loss did not improve from 0.09891
Epoch 459/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0458 - acc: 0.9892 - val_loss: 0.1231 - val_acc: 0.9773

Epoch 00459: val_loss did not improve from 0.09891
Epoch 460/512
43442/43442 [==============================] - 9s 212us/step - loss: 0.0408 - acc: 0.9901 - val_loss: 0.1591 - val_acc: 0.9733

Epoch 00460: val_loss did not improve from 0.09891
Epoch 461/512
43442/43442 [==============================] - 9s 212us/step - loss: 0.0462 - acc: 0.9879 - val_loss: 0.1395 - val_acc: 0.9792

Epoch 00461: val_loss did not improve from 0.09891
Epoch 462/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0420 - acc: 0.9899 - val_loss: 0.1541 - val_acc: 0.9700

Epoch 00462: val_loss did not improve from 0.09891
Epoch 463/512
43442/43442 [=====================

43442/43442 [==============================] - 9s 217us/step - loss: 0.0453 - acc: 0.9889 - val_loss: 0.1360 - val_acc: 0.9762

Epoch 00500: val_loss did not improve from 0.09891
Epoch 501/512
43442/43442 [==============================] - 9s 214us/step - loss: 0.0409 - acc: 0.9899 - val_loss: 0.1214 - val_acc: 0.9797

Epoch 00501: val_loss did not improve from 0.09891
Epoch 502/512
43442/43442 [==============================] - 9s 211us/step - loss: 0.0431 - acc: 0.9902 - val_loss: 0.1279 - val_acc: 0.9766

Epoch 00502: val_loss did not improve from 0.09891
Epoch 503/512
43442/43442 [==============================] - 9s 213us/step - loss: 0.0508 - acc: 0.9886 - val_loss: 0.1113 - val_acc: 0.9801

Epoch 00503: val_loss did not improve from 0.09891
Epoch 504/512
43442/43442 [==============================] - 9s 216us/step - loss: 0.0444 - acc: 0.9896 - val_loss: 0.1298 - val_acc: 0.9785

Epoch 00504: val_loss did not improve from 0.09891
Epoch 505/512
43442/43442 [======================

In [15]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
5431/5431 [==============================] - 1s 113us/step
Test Accuracy: 97.91935186890075


In [21]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

[INFO] Saving model...
